In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 1. Preparing Data to be Used in Machine learning

This may involve a few steps:
    1. Split data into `features` & `labels` - `X` and `Y`.
    2. Filling (`imputing`) or disregard missing values.
    3. converting non-numerical to numerical values (`feature encoding`)

### Separate the data into Questions & Answers

In this case, the question data will be all info about the heart desease diagnostics and the answer will be the `target` column which tells whether or not the patient is sick.

In [14]:
heart_desease = pd.read_csv('data/heart-disease.csv')
x = heart_desease.drop('target', axis=1)
y = heart_desease['target']


### Split both data Into Random Train & Test Data

Here we make use of `train_test_split`. When called, it returns the x test & train data and same for y.

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [16]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

### Convert Non-mumerical categories to numerical.

Here any column that could be used to group the data into rows with that similar attribute are converted to numerics as we'll see below. Let's try and predict car price based on some car features.

In [25]:
car_data = pd.read_csv('data/car-sales-extended.csv')
car_data.head()

x = car_data.drop('Price', axis=1)
y = car_data['Price']
car_data.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043


In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['Make', 'Colour', 'Doors']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot', one_hot,categorical_features)], remainder='passthrough')

transformed_x = transformer.fit_transform(x)
transformed_x


array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

In [28]:
transformed_x_pd = pd.DataFrame(transformed_x)
transformed_x_pd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0


To visualize what just happened, let's use `dummies`.

In [29]:
dummies = pd.get_dummies(car_data[['Make', 'Colour', 'Doors']])
dummies

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White
0,4,False,True,False,False,False,False,False,False,True
1,5,True,False,False,False,False,True,False,False,False
2,4,False,True,False,False,False,False,False,False,True
3,4,False,False,False,True,False,False,False,False,True
4,3,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
995,4,False,False,False,True,True,False,False,False,False
996,3,False,False,True,False,False,False,False,False,True
997,4,False,False,True,False,False,True,False,False,False
998,4,False,True,False,False,False,False,False,False,True


Therefore the transformations happen in some sort of truth table generation for each possible value of the categorical columns and a `1 or True` is put where the particular row is true else `0 or False` showing the row does not contain that attribute.

Without the transformation, the model won't be able t fit. See below.

In [31]:
from sklearn.ensemble import RandomForestRegressor # Regressor is best for number predictions
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
model = RandomForestRegressor()
model.fit(x_train, y_train)

ValueError: could not convert string to float: 'Honda'

But with the transformed data:

In [33]:
model.fit(transformed_x, y)

RandomForestRegressor()

In [35]:
x_train, x_test, y_train, y_test = train_test_split(transformed_x, y, test_size=0.2)
model.score(x_test, y_test)

0.8787326495836101

### Remove or Imputate Missing values

The rows with missing values can either be filled with some dummy data or be removed completely since the ML model can only work with complete data. Either way has it's advantages and disadvantages therefore it all comes down to the situation and personal preferences.

In [40]:
car_sales_missing = pd.read_csv('data/car-sales-extended-missing-data.csv')

# Check the missing data
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [39]:
car_sales_missing

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


Therefore an error will be produced if you try to transform the data:

In [45]:
x = car_sales_missing.drop('Price', axis=1)
y = car_sales_missing['Price']

categorical_features = ['Make', 'Colour', 'Doors']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot', one_hot,categorical_features)], remainder='passthrough')

transformed_x = transformer.fit_transform(x)
transformed_x

<1000x16 sparse matrix of type '<class 'numpy.float64'>'
	with 4000 stored elements in Compressed Sparse Row format>

#### a) FIll in the missing values using Pandas

In [49]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [52]:
car_sales_missing['Make'].fillna('missing', inplace=True)
car_sales_missing['Colour'].fillna('missing', inplace=True)
car_sales_missing['Doors'].fillna(4, inplace=True)
car_sales_missing['Odometer (KM)'].fillna(car_sales_missing['Odometer (KM)'].mean(), inplace=True)

In [53]:
car_sales_missing.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

In [54]:
# Drop all records without a Price since it is the one to be determined by the model
car_sales_missing.dropna(inplace=True)

In [55]:
car_sales_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

#### b) Fill missing data using Scikit-Learn

In [56]:
car_sales_missing = pd.read_csv('data/car-sales-extended-missing-data.csv')

# Drop rows with no label
car_sales_missing.dropna(subset='Price', inplace=True)
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [57]:
x = car_sales_missing.drop('Price', axis=1)
y = car_sales_missing.Price

In [60]:
# Use impute to fill data
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Fill categorical values with value 'missing' and numerical ones with mean
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
door_imputer = SimpleImputer(strategy='constant', fill_value=4)
num_imputer = SimpleImputer() # Default strategy is mean

# Define columns
cat_features = ['Make', 'Colour']
door_features = ['Doors']
num_features = ['Odometer (KM)']

# Create imputer - Something that fills missing values
imputer = ColumnTransformer([
    ('cat_imputer', cat_imputer, cat_features),
    ('door_imputer', door_imputer, door_features),
    ('num_imputer', num_imputer, num_features)
])

# Transform the data
filled_x = imputer.fit_transform(x)
filled_x

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)